# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../output_data/cities.csv"
df=pd.read_csv("../output_data/cities.csv")
df.head()



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     548 non-null    int64  
 1   City        548 non-null    object 
 2   Cloudiness  548 non-null    int64  
 3   Country     544 non-null    object 
 4   Date        548 non-null    int64  
 5   Humidity    548 non-null    int64  
 6   Lat         548 non-null    float64
 7   Lng         548 non-null    float64
 8   Max Temp    548 non-null    float64
 9   Wind Speed  548 non-null    float64
dtypes: float64(4), int64(4), object(2)
memory usage: 42.9+ KB


In [6]:
#convert humidity to float
# humidity = df['Humidity'].astype('float')
# humidity.head()

0    72.0
1    74.0
2    73.0
3    39.0
4    76.0
Name: Humidity, dtype: float64

In [14]:
locations = df[['Lat', 'Lng']]
locations.head()



,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42


In [15]:
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.
gmaps.configure(api_key=g_key)

fig = gmaps.figure(center=[20, 20], zoom_level=1)

layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=5)

fig.add_layer(layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
weather = df.loc[(df['Max Temp'] < 100) & (df['Max Temp'] > 70) 
                             & (df['Wind Speed'] < 10) & (df['Cloudiness'] == 0)]
weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# Create dataframe with columns: City, Country, Lat, Lng 
hotel_df = weather[['City', 'Country', 'Lat', 'Lng']].reset_index(drop=True)

# adding a "Hotel Name" column 
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,opuwo,NaN,-18.06,13.84,
1,colonelganj,IN,27.13,81.70,
2,ponta do sol,BR,-20.63,-46.00,
3,cidreira,BR,-30.17,-50.22,
4,arraial do cabo,BR,-22.97,-42.02,
5,vaini,IN,15.34,74.49,
6,sarai mir,IN,26.02,82.92,
7,nador,MA,35.17,-2.93,
8,mogok,MM,22.92,96.51,
9,kirkuk,IQ,35.47,44.40,


In [24]:
# Set parameters to search for a hotel,
params = {
        'radius': 5000,
        'type': 'lodging',
        'key': g_key
            }
    
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#loop thru hotel to get hotels
for index,row in hotel_df.iterrows():
    
    lat= row['Lat']
    lng= row['Lng']
    params["location"] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"] = response['results'][0]["name"]
    except: 
        print("hotel not found")
        
            
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,opuwo,NaN,-18.06,13.84,ABBA Guesthouse
1,colonelganj,IN,27.13,81.70,जय मां दुर्गा मंदिर
2,ponta do sol,BR,-20.63,-46.00,Hotel Escarpas do Lago
3,cidreira,BR,-30.17,-50.22,Hotel Castelo
4,arraial do cabo,BR,-22.97,-42.02,Pousada Porto Praia
5,vaini,IN,15.34,74.49,Dandeli Lake County
6,sarai mir,IN,26.02,82.92,Rasoolpur Barwa
7,nador,MA,35.17,-2.93,Airport hotel
8,mogok,MM,22.92,96.51,MOGOK HILL HOTEL
9,kirkuk,IQ,35.47,44.40,Bayaz Han (Beyaz Han)


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))